In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from skimage import io
import numpy as np
import os
import re
from PIL import Image

#setup a standard image size; this will distort some images but will get everything into the same shape
STANDARD_SIZE = (300, 167)

In [5]:

def read_img_to_matrix(filename, verbose=False):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    if verbose==True:
        #print ("changing size from %s to %s" % (str(img.size), str(STANDARD_SIZE)))
        img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    #img = map(list, img)
    img = np.array(img)
    return img

def flatten_image(img):
    """
    takes in an (m, n) numpy array and flattens it 
    into an array of shape (1, m * n)
    """
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

In [6]:
img_dir = "../roof_images/"
images = [img_dir+ f for f in os.listdir(img_dir)]

count=0
dic={}
#print(os.path.dirname(os.path.abspath(__file__)))

for image in images:
    #if count < 10 :
    count=count+1
    # By using skimage.io.imread, we load the image into numpy.ndarray format.
    #myfile=open(os.path.join(root,filename) , "rb")
    #img=io.imread(myfile, as_grey = True)
    img = read_img_to_matrix(image, verbose = True)
    img = flatten_image(img)

    # We keep only the id for each picture, so that we remove '.jpg' in the filename.
    filename=re.sub(r'../roof_images/','',re.sub(r'.jpg','',image))

    dic[filename]=img
    
    if count%5000==0:
        print(len(dic))

print(count)

img_data = pd.DataFrame(sorted(dic.items()))
img_data.columns = ['Id', 'data']
# Convert the Id type into string to ensure that it will match the Id of label
img_data['Id']=img_data['Id'].astype(str)
data_np = []

label=pd.read_csv("../data/id_train.csv")
# Convert the Id type into string to ensure that it will match the Id of img_data
label['Id']=label['Id'].astype(str)

# By merging the two datasets, we obtain the train set. There are 8000 observations with the label of roof type. 
#Since we have 42759 images, but only 8000 labels. It's possible to try semi-supervised learning with the unlabeled images.

train=pd.merge(img_data, label, on='Id', how='inner')
for value in train["data"].values:
    data_np.append(value)

print(train)
print(data_np)




5000
10000
15000
20000
25000
30000
35000
40000
42759


MemoryError: 

In [21]:
print(train)
print(data_np)

             Id                                               data  label
0      -1191173  [8, 23, 42, 10, 25, 44, 12, 27, 46, 13, 28, 47...      4
1      -1304053  [16, 40, 24, 16, 40, 24, 13, 37, 21, 17, 39, 2...      3
2      -1311082  [34, 35, 39, 34, 35, 39, 34, 35, 39, 29, 30, 3...      2
3      -1582718  [97, 131, 107, 97, 131, 107, 96, 130, 106, 95,...      3
4      -1586982  [49, 31, 43, 49, 31, 43, 49, 31, 43, 35, 22, 3...      4
5      -1813560  [10, 37, 56, 10, 37, 56, 10, 37, 56, 10, 37, 5...      3
6      -1937877  [98, 108, 83, 98, 108, 83, 90, 100, 75, 83, 93...      3
7      -1964319  [156, 166, 167, 156, 166, 167, 156, 166, 167, ...      3
8      -1981985  [121, 127, 125, 121, 127, 125, 119, 125, 123, ...      2
9      -2343303  [26, 49, 65, 26, 49, 65, 26, 47, 64, 26, 47, 6...      4
10     -2355684  [36, 45, 40, 36, 45, 40, 36, 45, 40, 32, 41, 3...      3
11     -2815459  [80, 88, 99, 80, 88, 99, 87, 95, 106, 87, 95, ...      3
12     -3935637  [29, 44, 39, 29, 44, 

In [30]:
train.to_csv("../data/train.csv", sep=',', index=False)

del img_data
del dic
del ordered

In [ ]:

from sklearn.decomposition import RandomizedPCA
import pylab as pl
pca = RandomizedPCA(n_components=2)
X = pca.fit_transform(np.array(data_np))
print(X)


df = pd.DataFrame({"x": X[:, 0], "y": X[:, 1]})
df["label"] = train["label"]
colors = ["red", "yellow", "green" , "blue"]
for label, color in zip(df['label'].unique(), colors):
    mask = df['label']==label
    pl.scatter(df[mask]['x'], df[mask]['y'], c=color, label=label)
pl.legend()
pl.show()

In [ ]:
del label
del images